In [1]:
import os
import sys
sys.path.append('../')
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import core.config as conf

In [2]:
path = conf.data_path

In [4]:
df = pd.read_csv('../model/notebooks/result/train_past_d.csv')

In [5]:
df.head()

,act_id,iem_cd,byn_dt,hold_d,sex_dit_cd,cus_age_stn_cd,ivs_icn_cd,cus_aet_stn_cd,mrz_pdt_tp_sgm_cd,lsg_sgm_cd,tco_cus_grd_cd,tot_ivs_te_sgm_cd,mrz_btp_dit_cd,iem_krl_nm,btp_cfc_cd,mkt_pr_tal_scl_tp_cd,stk_dit_cd,past_d,hist_d
0,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A006360,20180726,11,1,9,3,2,2,9,5,5,8,101,1,1,1,195.0,6.6
1,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005930,20180131,80,1,9,3,2,2,9,5,5,8,1361,9,1,1,86.0,48.0
2,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005070,20180517,5,1,9,3,2,2,9,5,5,8,2530,12,2,99,6.0,3.0
3,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A003520,20201112,22,1,9,3,2,2,9,5,5,8,1969,8,2,1,22.0,13.2
4,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A002310,20180905,324,1,9,3,2,2,9,5,5,8,1696,10,3,99,347.0,194.4


In [6]:
df.shape

(681472, 19)

In [7]:
def to_integer(dt_time):
    return 10000*dt_time.year + 100*dt_time.month + dt_time.day

In [8]:
def sell_day(buy, hold_d) :
    buy_d = str(buy)
    buy_d = datetime(int(buy_d[:4]), int(buy_d[4:6]), int(buy_d[6:]))
    sell_d = to_integer(buy_d+timedelta(days=hold_d))
    return sell_d

In [9]:
def working_day(s_dt, e_dt) : # 공휴일 제외
    s_dt = str(s_dt).split('.')[0]
    e_dt = str(e_dt).split('.')[0]
    s_dt = s_dt[:4] + '-' + s_dt[4:6] + '-' + s_dt[6:]
    e_dt = e_dt[:4] + '-' + e_dt[4:6] + '-' + e_dt[6:]
    return np.busday_count(s_dt, e_dt, weekmask='1111100')

In [10]:
df['sell_d'] = df.apply(lambda x : sell_day(x['byn_dt'], x['hold_d']), axis = 1)

In [11]:
tmp_train = df[df['sell_d'] < 20200601]

In [12]:
tmp_test = df[df['sell_d'] >= 20200601][df['byn_dt']<20200601]

/home/nyongja/anaconda3/envs/nh2021/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [13]:
tmp_test['hist_d'] = tmp_test.apply(lambda x : working_day(x['byn_dt'], 20200601), axis = 1 )

In [14]:
tmp_test

,act_id,iem_cd,byn_dt,hold_d,sex_dit_cd,cus_age_stn_cd,ivs_icn_cd,cus_aet_stn_cd,mrz_pdt_tp_sgm_cd,lsg_sgm_cd,tco_cus_grd_cd,tot_ivs_te_sgm_cd,mrz_btp_dit_cd,iem_krl_nm,btp_cfc_cd,mkt_pr_tal_scl_tp_cd,stk_dit_cd,past_d,hist_d,sell_d
7,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A000400,20200512,22,1,9,3,2,2,9,5,5,8,1112,2,2,99,22.0,14,20200603
35,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A046440,20200529,10,1,9,3,2,2,9,5,5,8,228,13,2,99,10.0,1,20200608
94,b6623032ed365b6c9a71c0344ebb8ca191f1ebc5927771...,A004800,20200511,83,1,6,0,3,2,5,5,6,9,3037,2,2,1,84.0,15,20200802
109,b6623032ed365b6c9a71c0344ebb8ca191f1ebc5927771...,A104460,20200528,63,1,6,0,3,2,5,5,6,9,1065,10,2,99,64.0,2,20200730
114,b6623032ed365b6c9a71c0344ebb8ca191f1ebc5927771...,A064960,20200528,55,1,6,0,3,2,5,5,6,9,509,6,2,1,55.0,2,20200722
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
662036,a76233bb62b570cf45ae7614c4607aff35a1807ecf4e89...,A226490,20200529,144,1,7,4,6,3,5,2,6,9,370,14,99,99,149.0,1,20201020
662039,faa16850888d4d3d6b423420a81225aa03062057247e65...,A005930,20200529,4,1,3,3,4,2,3,4,3,9,1361,9,1,1,49.5,1,20200602
662040,faa16850888d4d3d6b423420a81225aa03062057247e65...,A005389,20200529,10,1,3,3,4,2,3,4,3,9,3000,6,99,99,10.0,1,20200608
662058,faa16850888d4d3d6b423420a81225aa03062057247e65...,A041830,20200529,101,1,3,3,4,2,3,4,3,9,2238,10,2,99,106.0,1,20200907


In [15]:
tmp_test.to_csv(path + 'tmp_test_6.csv', index = False)

In [16]:
tmp_train.to_csv(path + 'tmp_train_6.csv', index = False)